In [1]:
%%capture
pip install transformers sentence_transformers openai

# Overview

In this tutorial, we'll use Feast to inject documents into the context of an LLM (Large Language Model) to power a RAG Application (Retrieval Augmented Generation) with Milvus as the online vector database.

Feast solves several common issues in this flow:
1. **Online retrieval:** At inference time, LLMs often need access to data that isn't readily 
   available and needs to be precomputed from other data sources.
2. **Vector Search:** Feast has built support for vector similarity search that is easily configured declaritively so users can focus on their application. Milvus provides powerful and efficient vector similarity search capabilities.
3. **Richer structured data:** Along with vector search, users can query standard structured fields to inject into the LLM context for better user experiences.
4. **Feature/Context and versioning:** Different teams within an organization are often unable to reuse 
   data across projects and services, resulting in duplicate application logic. Models have data dependencies that need 
   to be versioned, for example when running A/B tests on model/prompt versions.
   * Feast enables discovery of and collaboration on previously used documents, features, and enables versioning of sets of 
     data.

We will:
1. Deploy a local feature store with a **Parquet file offline store** and **Sqlite online store**.
2. Write/materialize the data (i.e., feature values) from the offline store (a parquet file) into the online store (Sqlite).
3. Serve the features using the Feast SDK with Milvus's vector search capabilitie
4. Inject the document into the LLM's context to answer questions

In [4]:
%%capture
! pip install feast[nlp] -U -q
! echo "Please restart your runtime now (Runtime -> Restart runtime). This ensures that the correct dependencies are loaded."

**Reminder**: Please restart your runtime after installing Feast (Runtime -> Restart runtime). This ensures that the correct dependencies are loaded.

## Step 2: Create a feature repository

A feature repository is a directory that contains the configuration of the feature store and individual features. This configuration is written as code (Python/YAML) and it's highly recommended that teams track it centrally using git. See [Feature Repository](https://docs.feast.dev/reference/feature-repository) for a detailed explanation of feature repositories.

The easiest way to create a new feature repository to use the `feast init` command in your terminal. For this RAG demo, you **do not** need to initialize a feast repo. We have already provided a complete feature repository for you in the current directory (check `feature_repo`) with all the necessary Milvus configurations set up and ready to use.


### Demo data scenario 
- We take data from the popular library [Docling](https://github.com/docling-project/docling) to parse PDFs into sentences which are used for RAG.
- Our goal is to show how simple it is to transform PDFs into text data that can be used for RAG applications with Milvus and Feast.

In [1]:
import feast
import warnings

warnings.filterwarnings('ignore')

### Step 2a: Inspecting the feature repository

Let's take a look at the demo repo itself. It breaks down into


* `data/` contains raw demo parquet data
* `example_repo.py` contains demo feature definitions
* `feature_store.yaml` contains a demo setup configuring where data sources are
* `test_workflow.py` showcases how to run all key Feast commands, including defining, retrieving, and pushing features.
   * You can run this with `python test_workflow.py`.

In [2]:
%cd feature_repo/
!ls -R

/Users/farceo/dev/feast/examples/rag-docling/feature_repo
__pycache__          example_repo.py      transformed_rows.pkl
data                 feature_store.yaml

./__pycache__:
example_repo.cpython-310.pyc example_repo.cpython-311.pyc

./data:
docling_samples.parquet       small.pdf
metadata_samples.parquet      smallest-possible-pdf-2.0.pdf
online_store.db


### Step 2b: Inspecting the project configuration
Let's inspect the setup of the project in `feature_store.yaml`. 

The key line defining the overall architecture of the feature store is the **provider**. 

The provider value sets default offline and online stores. 
* The offline store provides the compute layer to process historical data (for generating training data & feature 
  values for serving). 
* The online store is a low latency store of the latest feature values (for powering real-time inference).

Valid values for `provider` in `feature_store.yaml` are:

* local: use file source with Milvus Lite
* gcp: use BigQuery/Snowflake with Google Cloud Datastore/Redis
* aws: use Redshift/Snowflake with DynamoDB/Redis

Note that there are many other offline / online stores Feast works with, including Azure, Hive, Trino, and PostgreSQL via community plugins. See https://docs.feast.dev/roadmap for all supported connectors.

A custom setup can also be made by following [Customizing Feast](https://docs.feast.dev/v/master/how-to-guides/customizing-feast)

In [3]:
!pygmentize feature_store.yaml

project: rag
provider: local
registry: data/registry.db
online_store:
  type: milvus
  path: data/online_store.db
  embedding_dim: 384
  index_type: "FLAT"

offline_store:
  type: file
entity_key_serialization_version: 3
auth:
    type: no_auth


### Inspecting the raw data

The raw feature data we have in this demo is stored in a local parquet file. The dataset Wikipedia summaries of diferent cities.

In [4]:
import pandas as pd 

df = pd.read_parquet("./data/docling_samples.parquet")
mdf = pd.read_parquet("./data/metadata_samples.parquet")
df['chunk_embedding'] = df['vector'].apply(lambda x: x.tolist())
embedding_length = len(df['vector'][0])
print(f'embedding length = {embedding_length}')

embedding length = 384


In [5]:
df['created'] = pd.Timestamp.now()
mdf['created'] = pd.Timestamp.now()

In [6]:
from IPython.display import display

display(df.head())

,document_id,chunk_id,file_name,raw_chunk_markdown,vector,chunk_embedding,created
0,doc-1,chunk-1,2203.01017v2,"Ahmed Nassar, Nikolaos Livathinos, Maksym Lysa...","[-0.056879762560129166, 0.01667858101427555, -...","[-0.056879762560129166, 0.01667858101427555, -...",2025-04-20 23:19:48.930517
1,doc-1,chunk-2,2203.01017v2,a. Picture of a table:\nTables organize valuab...,"[0.050771258771419525, -0.0055733839981257915,...","[0.050771258771419525, -0.0055733839981257915,...",2025-04-20 23:19:48.930517
2,doc-1,chunk-3,2203.01017v2,a. Picture of a table:\ncomplex column/row-hea...,"[-0.05088765174150467, 0.05101901665329933, -0...","[-0.05088765174150467, 0.05101901665329933, -0...",2025-04-20 23:19:48.930517
3,doc-1,chunk-4,2203.01017v2,a. Picture of a table:\nmodel. The latter impr...,"[0.011835305020213127, -0.09409898519515991, 0...","[0.011835305020213127, -0.09409898519515991, 0...",2025-04-20 23:19:48.930517
4,doc-1,chunk-5,2203.01017v2,a. Picture of a table:\nwe can obtain the cont...,"[-0.0068757119588553905, 0.006624480709433556,...","[-0.0068757119588553905, 0.006624480709433556,...",2025-04-20 23:19:48.930517


In [7]:
display(mdf.head())

,document_id,file_name,full_document_markdown,pdf_bytes,created
0,doc-1,2203.01017v2,## TableFormer: Table Structure Understanding ...,b'%PDF-1.5\n%\x8f\n5 0 obj\n<< /Type /XObject ...,2025-04-20 23:19:48.931844
1,doc-3,2305.03393v1-pg9,order to compute the TED score. Inference timi...,b'%PDF-1.3\n%\xc4\xe5\xf2\xe5\xeb\xa7\xf3\xa0\...,2025-04-20 23:19:48.931844
2,doc-2,2305.03393v1,## Optimized Table Tokenization for Table Stru...,b'%PDF-1.5\n%\x8f\n74 0 obj\n<< /Filter /Flate...,2025-04-20 23:19:48.931844
3,doc-4,amt_handbook_sample,"pulleys, provided the inner race of the bearin...",b'%PDF-1.6\r%\xe2\xe3\xcf\xd3\r\n875 0 obj\r<<...,2025-04-20 23:19:48.931844
4,doc-5,code_and_formula,## JavaScript Code Example\n\nLorem ipsum dolo...,b'%PDF-1.5\n%\xbf\xf7\xa2\xfe\n3 0 obj\n<< /Li...,2025-04-20 23:19:48.931844


## Step 3: Register feature definitions and deploy your feature store

`feast apply` scans python files in the current directory for feature/entity definitions and deploys infrastructure according to `feature_store.yaml`.

### Step 3a: Inspecting feature definitions
Let's inspect what `example_repo.py` looks like:

```python
```

### Step 3b: Applying feature definitions
Now we run `feast apply` to register the feature views and entities defined in `example_repo.py`, and sets up SQLite online store tables. Note that we had previously specified SQLite as the online store in `feature_store.yaml` by specifying a `local` provider.

In [8]:
%rm -rf .ipynb_checkpoints/

In [9]:
! feast apply 

/Users/farceo/dev/feast/.venv/lib/python3.11/site-packages/pymilvus/client/__init__.py:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import DistributionNotFound, get_distribution
/Users/farceo/dev/feast/.venv/lib/python3.11/site-packages/pkg_resources/__init__.py:3147: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('sphinxcontrib')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/Users/farceo/dev/feast/.venv/lib/python3.11/site-packages/environs/__init__.py:58: DeprecationWarning: The '__version_info__' attribute is deprecated and will be removed in in a future version. Use feature detection or 'packaging.Version(importlib.metadata.version("marshmallow")).release' instead.
  _SUP

## Step 5: Load features into your online store

In [10]:
from datetime import datetime
from feast import FeatureStore

store = FeatureStore(repo_path=".")

### Step 5a: Using `write_to_online_store`

We now serialize the latest values of features since the beginning of time to prepare for serving. Note, `write_to_online_store` serializes all new features since the last `write_to_online_store` call, or since the time provided minus the `ttl` timedelta. 

In [11]:
df.head()

,document_id,chunk_id,file_name,raw_chunk_markdown,vector,chunk_embedding,created
0,doc-1,chunk-1,2203.01017v2,"Ahmed Nassar, Nikolaos Livathinos, Maksym Lysa...","[-0.056879762560129166, 0.01667858101427555, -...","[-0.056879762560129166, 0.01667858101427555, -...",2025-04-20 23:19:48.930517
1,doc-1,chunk-2,2203.01017v2,a. Picture of a table:\nTables organize valuab...,"[0.050771258771419525, -0.0055733839981257915,...","[0.050771258771419525, -0.0055733839981257915,...",2025-04-20 23:19:48.930517
2,doc-1,chunk-3,2203.01017v2,a. Picture of a table:\ncomplex column/row-hea...,"[-0.05088765174150467, 0.05101901665329933, -0...","[-0.05088765174150467, 0.05101901665329933, -0...",2025-04-20 23:19:48.930517
3,doc-1,chunk-4,2203.01017v2,a. Picture of a table:\nmodel. The latter impr...,"[0.011835305020213127, -0.09409898519515991, 0...","[0.011835305020213127, -0.09409898519515991, 0...",2025-04-20 23:19:48.930517
4,doc-1,chunk-5,2203.01017v2,a. Picture of a table:\nwe can obtain the cont...,"[-0.0068757119588553905, 0.006624480709433556,...","[-0.0068757119588553905, 0.006624480709433556,...",2025-04-20 23:19:48.930517


## Ingesting transformed data to the feature view that has no associated transformation

In [12]:
store.write_to_online_store(feature_view_name='docling_feature_view', df=df)

Connecting to Milvus in local mode using data/online_store.db


## Ingesting Pre-transformed data that was created in our Docling Demo notebook

In [13]:
# Turning off transformation on writes is as simple as changing the default behavior
store.write_to_online_store(
    feature_view_name='docling_transform_docs', 
    df=df[df['document_id']!='doc-1'], 
    transform_on_write=False,
)


## Ingesting the raw data data and transforming before insertion to Milvus with Docling

In [14]:
# Now we can transform a raw PDF on the fly
store.write_to_online_store(
    feature_view_name='docling_transform_docs', 
    df=mdf[mdf['document_id']=='doc-1'], 
    transform_on_write=True, # this is the default
)

Token indices sequence length is longer than the specified maximum sequence length for this model (933 > 512). Running this sequence through the model will result in indexing errors


### Step 5b: Inspect materialized features

Note that now there are `online_store.db` and `registry.db`, which store the materialized features and schema information, respectively.

In [15]:
pymilvus_client = store._provider._online_store._connect(store.config)
COLLECTION_NAME = [c for c in pymilvus_client.list_collections() if 'docling_transform_docs' in c][0]

milvus_query_result = pymilvus_client.query(
    collection_name=COLLECTION_NAME,
    filter="document_id == 'doc-1'",
    limit=1000,
)
pd.DataFrame(milvus_query_result).head()

,document_id_chunk_id_pk,chunk_id,chunk_text,created_ts,document_id,event_ts,vector
0,0200000002000000080000006368756e6b5f6964020000...,chunk-0,"Ahmed Nassar, Nikolaos Livathinos, Maksym Lysa...",1745220099533292,doc-1,1745220099533292,"[-0.056879763, 0.016678581, -0.019722786, -0.0..."
1,0200000002000000080000006368756e6b5f6964020000...,chunk-1,a. Picture of a table:\nTables organize valuab...,1745220099533294,doc-1,1745220099533294,"[0.05077126, -0.005573384, -0.05867869, 0.0341..."
2,0200000002000000080000006368756e6b5f6964020000...,chunk-2,a. Picture of a table:\ncomplex column/row-hea...,1745220099533295,doc-1,1745220099533295,"[-0.05088765, 0.051019017, -0.06598652, -0.045..."
3,0200000002000000080000006368756e6b5f6964020000...,chunk-3,a. Picture of a table:\nmodel. The latter impr...,1745220099533295,doc-1,1745220099533295,"[0.011835305, -0.094098985, 0.00086131715, -0...."
4,0200000002000000080000006368756e6b5f6964020000...,chunk-4,a. Picture of a table:\nwe can obtain the cont...,1745220099533295,doc-1,1745220099533295,"[-0.006875712, 0.0066244807, -0.10691858, -0.0..."


### Quick note on entity keys
Note from the above command that the online store indexes by `entity_key`. 

[Entity keys](https://docs.feast.dev/getting-started/concepts/entity#entity-key) include a list of all entities needed (e.g. all relevant primary keys) to generate the feature vector. In this case, this is a serialized version of the `document_id`. We use this later to fetch all features for a given driver at inference time.

## Step 6: Embedding a query using PyTorch and Sentence Transformers

During inference (e.g., during when a user submits a chat message) we need to embed the input text. This can be thought of as a feature transformation of the input data. In this example, we'll do this with a small Sentence Transformer from Hugging Face.

In [16]:
from example_repo import embed_text

In [17]:
embed_text("this is an example sentence")[0:10]

[0.06765689700841904,
 0.06349590420722961,
 0.0487130805850029,
 0.07930495589971542,
 0.03744804859161377,
 0.0026527801528573036,
 0.039374902844429016,
 -0.007098457310348749,
 0.05936148017644882,
 0.031537000089883804]

## Step 7: Fetching real-time vectors and data for online inference

At inference time, we need to use vector similarity search through the document embeddings from the online feature store using `retrieve_online_documents_v2()` while passing the embedded query. These feature vectors can then be fed into the context of the LLM.

In [18]:
sample_query = df['raw_chunk_markdown'].values[0] 
print(sample_query)

Ahmed Nassar, Nikolaos Livathinos, Maksym Lysak, Peter Staar IBM Research
{ ahn,nli,mly,taa @zurich.ibm.com }


In [19]:
# Note we can enhance this special case to embed within the feature server, optionally.
query_embedding = embed_text(sample_query)

### Let's fetch the data from the "batch" version of the documents stored in the `docling_feature_view` FeatureView

In [20]:
# Retrieve top k documents
context_data = store.retrieve_online_documents_v2(
    features=[
        "docling_feature_view:vector",
        "docling_feature_view:file_name",
        "docling_feature_view:raw_chunk_markdown",
        "docling_feature_view:chunk_id",
    ],
    query=query_embedding,
    top_k=3,
    distance_metric='COSINE',
).to_df()

display(context_data)

,vector,file_name,raw_chunk_markdown,chunk_id,distance
0,"[-0.056879762560129166, 0.01667858101427555, -...",2203.01017v2,"Ahmed Nassar, Nikolaos Livathinos, Maksym Lysa...",chunk-1,1.000000
1,"[-0.056879762560129166, 0.01667858101427555, -...",2203.01017v2,"References\n[1] Nicolas Carion, Francisco Mass...",chunk-188,0.370859
2,"[-0.056879762560129166, 0.01667858101427555, -...",2203.01017v2,2. Previous work and State of the Art\nhand. H...,chunk-31,0.352598


### Now let's fetch the data from the "on demand" version of the documents stored in the `docling_transform_docs` FeatureView

In [21]:
# Retrieve top k documents from the transformed data
context_data = store.retrieve_online_documents_v2(
    features=[
        "docling_transform_docs:vector",
        "docling_transform_docs:document_id",
        "docling_transform_docs:chunk_text",
        "docling_transform_docs:chunk_id",
    ],
    query=query_embedding,
    top_k=3,
    distance_metric='COSINE',
).to_df()

display(context_data)

,vector,document_id,chunk_text,chunk_id,distance
0,"[-0.056879762560129166, 0.01667858101427555, -...",doc-1,"Ahmed Nassar, Nikolaos Livathinos, Maksym Lysa...",chunk-0,NaN
1,"[-0.056879762560129166, 0.01667858101427555, -...",doc-7,,chunk-25,0.978799
2,"[-0.056879762560129166, 0.01667858101427555, -...",doc-7,,chunk-72,0.968456


### `FeatureView` vs. `OnDemandFeatureView` for Vector Search

If you look in `example_repo.py` you'll notice that `docling_example_feature_view` and `docling_transform_docs` are very similar
with the exception of `docling_transform_docs` having the schema defined in the `@on_demand_feature_view` decorator and a function 
(i.e., a feature transformation) implemented after the name declaration.

On the backend, Feast orchestrates the execution of this transformation within the Feature Server so that Feast can transform your 
documents with Docling via API and make your docs available for vector similarity search after transformation and insertion to the online store.

In [22]:
 def format_documents(context_df):
    output_context = ""
    
    # Remove duplicates based on 'chunk_id' (ensuring unique document chunks)
    unique_documents = context_df.drop_duplicates(subset=["chunk_id"])["chunk_text"]
    
    # Format each document
    for i, document_text in enumerate(unique_documents):
        output_context += f"****START DOCUMENT {i}****\n"
        output_context += f"document = {{ {document_text.strip()} }}\n"
        output_context += f"****END DOCUMENT {i}****\n\n"
    
    return output_context.strip()

In [23]:
RAG_CONTEXT = format_documents(context_data)

In [24]:
print(RAG_CONTEXT)

****START DOCUMENT 0****
document = { Ahmed Nassar, Nikolaos Livathinos, Maksym Lysak, Peter Staar IBM Research
{ ahn,nli,mly,taa @zurich.ibm.com } }
****END DOCUMENT 0****

****START DOCUMENT 1****
document = {  }
****END DOCUMENT 1****

****START DOCUMENT 2****
document = {  }
****END DOCUMENT 2****


In [25]:
FULL_PROMPT = f"""
You are an assistant for answering questions about a series of documents. You will be provided documentation from different documents. Provide a conversational answer.
If you don't know the answer, just say "I do not know." Don't make up an answer.

Here are document(s) you should use when answer the users question:
{RAG_CONTEXT}
"""

In [26]:
question = 'Who are the authors of the paper?'

In [27]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [28]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": FULL_PROMPT},
        {"role": "user", "content": question}
    ],
)

In [29]:
print('\n'.join([c.message.content for c in response.choices]))

The authors of the paper are Ahmed Nassar, Nikolaos Livathinos, Maksym Lysak, and Peter Staar from IBM Research.


# End